# LAB 3 Bayesian Regression: A Comparsion

Introduction
Reference: Yves Hilpisch: "Python for Finance" Chapter 11. 2015.

This laborary is to demonstrate the difference between classical regression and Baynesian regression.  

With PyMC3 the Python ecosystem provides a powerful and performant library to technically implement Bayesian statistics. PyMC3 is already a powerful library at the time of this writing. However, it is still in its early stages, so you should expect further enhancements, changes to the API, etc. 

As a benchmark, consider first an ordinary least-squares regression given the noisy data, the result of the “standard” regression approach is fixed values for the parameters of the regression line. Whereas, the Bayesian regression approach, we assume that the parameters are distributed in a certain way. 

For example, consider the equation describing the regression line ŷ ( x ) = a + bx , it is assumed the following priors : 

a is normally distributed with mean 0 and a standard deviation of 20.
b is normally distributed with mean 0 and a standard deviation of 20.
For the likelihood , we assume a normal distribution with mean of ŷ ( x ) and a uniformly distributed standard deviation between 0 and 10. A major element of Bayesian regression is (Markov Chain) Monte Carlo (MCMC) sampling.  
In principle, this is the same as drawing balls multiple times from boxes, as in the previous simple example just in a more systematic, automated way. 

For the technical sampling, there are three different functions to call: "find_MAP" finds the starting point for the sampling algorithm by deriving the local maximum a posteriori point .

"NUTS" implements the so-called “efficient No-U-Turn Sampler with dual averaging” (NUTS) algorithm for MCMC sampling given the assumed priors.

"Sample" draws a number of samples given the starting value from find_MAP and the optimal step size from the NUTS algorithm.

All this is to be wrapped into a PyMC3 Model object and executed within an API: 
 

#### Import Library, PyMC3

In [ ]:
import warnings 
warnings.simplefilter('ignore') 
import pymc3 as pm 
import numpy as np 
np.random.seed(1000) 
import matplotlib.pyplot as plt 
%matplotlib inline
from scipy import stats
from statsmodels.formula.api import glm as glm_sm
import arviz as az
az.style.use('arviz-darkgrid')

### Baynesian Idea Exposition

Below it is a binormal random trial with different probability of success: 0.25, .5, 0.75

In [ ]:
n_params = [1, 2, 4]  # Number of trials
p_params = [0.25, 0.5, 0.75]  # Probability of success

x = np.arange(0, max(n_params)+1)
f,ax = plt.subplots(len(n_params), len(p_params), sharex=True, sharey=True,
                    figsize=(8, 7), constrained_layout=True)

for i in range(len(n_params)):
    for j in range(len(p_params)):
        n = n_params[i]
        p = p_params[j]

        y = stats.binom(n=n, p=p).pmf(x)

        ax[i,j].vlines(x, 0, y, colors='C0', lw=5)
        ax[i,j].set_ylim(0, 1)
        ax[i,j].plot(0, 0, label="N = {:3.2f}\nθ = {:3.2f}".format(n,p), alpha=0)
        ax[i,j].legend()

        ax[2,1].set_xlabel('y')
        ax[1,0].set_ylabel('p(y|θ,N)')
        ax[0,0].set_xticks(x)

plt.savefig('B11197_01_03.png', dpi=300)

#### Multiple Trial the Coin to detemine the unknown fairness

In [ ]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 8))

if __name__ == "__main__":
    # Create a list of the number of coin tosses ("Bernoulli trials")
    number_of_trials = [0, 2, 10, 20, 50, 500]

    # Conduct 500 coin tosses and output into a list of 0s and 1s
    # where 0 represents a tail and 1 represents a head
    data = stats.bernoulli.rvs(0.5, size=number_of_trials[-1])
    
    # Discretise the x-axis into 100 separate plotting points
    x = np.linspace(0, 1, 100)
    
    # Loops over the number_of_trials list to continually add
    # more coin toss data. For each new set of data, we update
    # our (current) prior belief to be a new posterior. This is
    # carried out using what is known as the Beta-Binomial model.
    # For the time being, we won't worry about this too much. It 
    # will be the subject of a later article!
    for i, N in enumerate(number_of_trials):
        # Accumulate the total number of heads for this 
        # particular Bayesian update
        heads = data[:N].sum()

        # Create an axes subplot for each update 
        ax = plt.subplot(len(number_of_trials) / 2, 2, i + 1)
        ax.set_title("%s trials, %s heads" % (N, heads))

        # Add labels to both axes and hide labels on y-axis
        plt.xlabel("$P(H)$, Probability of Heads")
        plt.ylabel("Density")
        if i == 0:
            plt.ylim([0.0, 2.0])
        plt.setp(ax.get_yticklabels(), visible=False)
                
        # Create and plot a  Beta distribution to represent the 
        # posterior belief in fairness of the coin.
        y = stats.beta.pdf(x, 1 + heads, 1 + N - heads)
        plt.plot(x, y, label="observe %d tosses,\n %d heads" % (N, heads))
        plt.fill_between(x, 0, y, color="#aaaadd", alpha=0.5)

    # Expand plot to cover full width/height and show it
    plt.tight_layout()
    plt.show()

##### Generate the Beta Distribution based on different hyperparameters of alpha and beta

In [ ]:
params = [0.5, 1, 2, 3]
x = np.linspace(0, 1, 100)
f, ax = plt.subplots(len(params), len(params), sharex=True, sharey=True,
                     figsize=(8, 7), constrained_layout=True)
for i in range(4):
    for j in range(4):
        a = params[i]
        b = params[j]
        y = stats.beta(a, b).pdf(x)
        ax[i,j].plot(x, y)
        ax[i,j].plot(0, 0, label="α = {:2.1f}\nβ = {:2.1f}".format(a, b), alpha=0)
        ax[i,j].legend()
ax[1,0].set_yticks([])
ax[1,0].set_xticks([0, 0.5, 1])
f.text(0.5, 0.05, 'θ', ha='center')
f.text(0.07, 0.5, 'p(θ)', va='center', rotation=0)
plt.savefig('B11197_01_04.png', dpi=300)

Starting from different belief of theta (based on different hyperparameters of alpha and beta to generate different prior different distribution (colour: orange, green and purple) of theta)

In [ ]:
plt.figure(figsize=(10, 8))

n_trials = [0, 1, 2, 3, 4, 8, 16, 32, 50, 150]
data = [0, 1, 1, 1, 1, 4, 6, 9, 13, 48]
theta_real = 0.35

beta_params = [(1, 1), (20, 20), (1, 4)]
dist = stats.beta
x = np.linspace(0, 1, 200)

for idx, N in enumerate(n_trials):
    if idx == 0:
        plt.subplot(4, 3, 2)
        plt.xlabel('θ')
    else:
        plt.subplot(4, 3, idx+3)
        plt.xticks([])
    y = data[idx]
    for (a_prior, b_prior) in beta_params:
        p_theta_given_y = dist.pdf(x, a_prior + y, b_prior + N - y)
        plt.fill_between(x, 0, p_theta_given_y, alpha=0.7)

    plt.axvline(theta_real, ymax=0.3, color='k')
    plt.plot(0, 0, label=f'{N:4d} trials\n{y:4d} heads', alpha=0)
    plt.xlim(0, 1)
    plt.ylim(0, 12)
    plt.legend()
    plt.yticks([])
plt.tight_layout()
plt.savefig('B11197_01_05.png', dpi=300)

###### One will find that as more trial samples, the posterior distributions of different priod distribution converge

#### Classical Regression

##### Data generation Process by y = 4 + 2*x + random noise

In [ ]:
x = np.linspace(0,10,500) 
y = 4 + 2*x + np.random.standard_normal(len(x))*2
# Linear regression
reg = np.polyfit(x,y,1)
print(f"sample size = ",len(x))

#### Plot Graph of Classical Rgression Based on Frequentist

In [ ]:
plt.figure(figsize = (8,4)) 
plt.scatter(x,y,c = y, marker= 'v') 
plt.plot(x,reg [1] + reg [0]*x, lw = 2.0) 
plt.colorbar() 
plt.grid (True) 
plt.xlabel('x') 
plt.ylabel('y') 
# tag: pm_fig_0
# title: Sample data points a regresion line
# size: 90

##### Regression Coefficients of the Classical Regression

In [ ]:
reg

#### Bayesian Regression

In [ ]:
with pm.Model() as model: 
        # model specifications in PyMC3
        # are wrapped in a with-statement
    # define priors
    alpha = pm.Normal('alpha', mu=0, sd=20)
    beta = pm.Normal('beta', mu=0, sd=20)
    sigma = pm.Uniform('sigma', lower=0, upper=10)
    
    # define linear regression
    y_est = alpha + beta * x
    
    # define likelihood
    likelihood = pm.Normal('y', mu=y_est, sd=sigma, observed=y)
    
    # inference
    start = pm.find_MAP()
      # find starting value by optimization
    step = pm.NUTS()
      # instantiate MCMC sampling algorithm
    trace = pm.sample(100, step, start=start, progressbar=True)
      # draw 100 posterior samples using NUTS sampling

In [ ]:
trace[0]

In [ ]:
fig = pm.plots.traceplot(trace)
plt.figure(figsize=(8, 8))
#tag: pm_fig_1
#title: Trace plots for alpha, beta and sigma
# size: 100

Plot the Bayesian regression result

In [ ]:
plt.figure(figsize=(8, 4))
plt.scatter(x, y, c=y, marker='v')
plt.colorbar()
plt.grid(True)
plt.xlabel('x')
plt.ylabel('y')
for i in range(len(trace)):
    plt.plot(x, trace['alpha'][i] + trace['beta'][i] * x)
# tag: pm_fig_2
# title: Sample data and regression lines from Bayesian regression
# size: 100

THE END